# Inferencia en SLEAP

Este notebook realiza el análisis de video utilizando DeepLabCut para el seguimiento de movimiento animal.

## Pasos del proceso:
1. Importación de librerías necesarias
2. Carga y verificación del archivo de configuración
3. Análisis de propiedades del video
4. Recorte del video a una región de interés (ROI)
5. Análisis del video con DeepLabCut
6. Filtrado de predicciones
7. Creación de video con etiquetas

## 1. Importación de librerías

Importamos las librerías necesarias para el procesamiento de video y análisis con DeepLabCut.

In [ ]:
import deeplabcut as dlc
import cv2
import numpy as np
import os
import sys
import yaml
import matplotlib.pyplot as plt
import time

print('Importación de módulos completada')

## 2. Carga del archivo de configuración

Cargamos el archivo de configuración de SLEAP/DeepLabCut y verificamos que todos los directorios necesarios existan.

In [ ]:
# Ruta al archivo de configuración
path_config_file = 'Experimento Olfato-JJ-2024-02-19/config2.yaml'

# Verificamos que el archivo de configuración exista
if not os.path.exists(path_config_file):
    print(f'ERROR: No existe el archivo de configuración en {path_config_file}')
    sys.exit(1)

print(f'Archivo de configuración encontrado: {path_config_file}')

# Cargamos el contenido del archivo de configuración
with open(path_config_file, 'r') as file:
    config = yaml.safe_load(file)

print('\nContenido de la configuración:')
print(config)

# Verificamos que el directorio del proyecto exista
project_path = config.get('project_path', '')
if not project_path:
    print('\nERROR: No se encontró project_path en la configuración')
    sys.exit(1)

if not os.path.exists(project_path):
    print(f'\nERROR: El directorio del proyecto no existe: {project_path}')
    sys.exit(1)

print(f'\nDirectorio del proyecto verificado: {project_path}')

## 3. Análisis de propiedades del video

Cargamos el video y extraemos sus propiedades principales (fps, número de frames, dimensiones, etc.).
También visualizamos un frame aleatorio para verificar que el video se cargó correctamente.

In [ ]:
# Ruta al video de entrada
video_path = 'experiments/A-7.mp4'

# Verificamos que el archivo de video exista
if not os.path.exists(video_path):
    print(f'ERROR: No existe el archivo de video en {video_path}')
    sys.exit(1)

print(f'Archivo de video encontrado: {video_path}')

# Abrimos el video para analizar sus propiedades
cap = cv2.VideoCapture(video_path)

# Verificamos que el video se haya abierto correctamente
if not cap.isOpened():
    print('ERROR: Error al abrir el video')
    sys.exit(1)

# Intentamos leer el primer frame
ret, frame = cap.read()
if not ret:
    print('ERROR: No se pudo leer el primer frame del video')
    cap.release()
    sys.exit(1)

print('Video abierto correctamente')

# Extraemos las propiedades básicas del video
fps = cap.get(cv2.CAP_PROP_FPS)
total_frames = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fourcc = int(cap.get(cv2.CAP_PROP_FOURCC))

# Mostramos la información del video
print(f'\nPropiedades del video:')
print(f'  Frames por segundo (FPS): {fps}')
print(f'  Total de frames: {total_frames}')
print(f'  Ancho: {width} píxeles')
print(f'  Alto: {height} píxeles')
print(f'  Codec (FOURCC): {fourcc}')

# Seleccionamos un frame aleatorio para visualizar
random_frame_number = np.random.randint(0, total_frames)
cap.set(cv2.CAP_PROP_POS_FRAMES, random_frame_number)
ret, random_frame = cap.read()

# Mostramos el frame seleccionado
if ret:
    plt.figure(figsize=(10, 6))
    frame_rgb = cv2.cvtColor(random_frame, cv2.COLOR_BGR2RGB)
    plt.imshow(frame_rgb)
    plt.axis('off')
    plt.title(f'Frame aleatorio #{random_frame_number}')
    plt.tight_layout()
    plt.show()
else:
    print(f'ADVERTENCIA: No se pudo leer el frame #{random_frame_number}')

# Liberamos el recurso del video
cap.release()
print('\nAnálisis de propiedades completado')

## 4. Recorte del video a una región de interés (ROI)

Recortamos el video a una región de interés específica para enfocarnos en el área relevante del experimento.
Esto reduce el tiempo de procesamiento y mejora la precisión del análisis.

In [ ]:
# Definimos la región de interés (ROI) para recortar el video
roi_x = 30      # Coordenada X inicial
roi_y = 70      # Coordenada Y inicial
roi_w = 520     # Ancho de la región
roi_h = 245     # Alto de la región

# Rutas de los archivos
video_path = 'experiments/A-7.mp4'
video_out = 'experiments/A-7_crop.mp4'

print(f'Recortando video: {video_path}')
print(f'Región de interés: x={roi_x}, y={roi_y}, ancho={roi_w}, alto={roi_h}')

# Abrimos el video de entrada
cap = cv2.VideoCapture(video_path)
if not cap.isOpened():
    print('ERROR: Error al abrir el video')
    sys.exit(1)

# Obtenemos las propiedades del video original
fps = cap.get(cv2.CAP_PROP_FPS)
frame_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

print(f'Video original: {frame_count} frames a {fps} FPS')

# Configuramos el codec para el video de salida
fourcc = cv2.VideoWriter_fourcc(*'mp4v')

# Creamos el escritor de video con las dimensiones del ROI
out = cv2.VideoWriter(video_out, fourcc, fps, (roi_w, roi_h))

# Procesamos cada frame del video
print(f'Procesando frames...')
frame_num = 0

while True:
    # Leemos el siguiente frame
    ret, frame = cap.read()
    
    # Si no hay más frames, terminamos
    if not ret:
        break
    
    # Recortamos el frame a la región de interés
    roi_frame = frame[roi_y:roi_y+roi_h, roi_x:roi_x+roi_w]
    
    # Guardamos el frame recortado
    out.write(roi_frame)
    
    # Incrementamos el contador
    frame_num += 1
    
    # Mostramos progreso cada 100 frames
    if frame_num % 100 == 0:
        print(f'  Procesados {frame_num}/{frame_count} frames...')

# Liberamos los recursos
cap.release()
out.release()

print(f'\nVideo recortado guardado exitosamente')
print(f'  Archivo: {video_out}')
print(f'  Frames procesados: {frame_num}')

## 5. Análisis del video con DeepLabCut

Ejecutamos el análisis del video recortado utilizando DeepLabCut para detectar y rastrear los puntos clave del animal.

In [ ]:
# Ruta al video que vamos a analizar
video_anal = 'experiments/A-7_crop.mp4'

print(f'Iniciando análisis con DeepLabCut...')
print(f'Video: {video_anal}')

# Registramos el tiempo de inicio
start_time = time.time()

# Ejecutamos el análisis con DeepLabCut
# Este proceso identifica y rastrea los puntos clave del animal en cada frame
dlc.analyze_videos(
    path_config_file,
    [video_anal],
    videotype='mp4',
    shuffle=1,
    trainingsetindex=0,
    gputouse=None,
    save_as_csv=True
)

# Calculamos el tiempo transcurrido
elapsed_time = time.time() - start_time
minutes = elapsed_time / 60

print(f'\nAnálisis completado exitosamente')
print(f'  Tiempo: {elapsed_time:.2f} segundos ({minutes:.2f} minutos)')

## 6. Filtrado de predicciones

Aplicamos un filtro ARIMA a las predicciones para suavizar las trayectorias y reducir el ruido en los datos de seguimiento.

In [ ]:
print('Aplicando filtro ARIMA a las predicciones...')
print('Este filtro suaviza las trayectorias y reduce el ruido en los datos')

# Parámetros del filtro ARIMA
window_length = 5    # Tamaño de la ventana temporal
ar_degree = 3        # Grado del componente autoregresivo
ma_degree = 1        # Grado del componente de media móvil
p_bound = 0.7        # Umbral de probabilidad
alpha = 0.01         # Nivel de significancia

# Aplicamos el filtro a las predicciones
dlc.filterpredictions(
    path_config_file,
    [video_anal],
    shuffle=1,
    trainingsetindex=0,
    filtertype='arima',
    windowlength=window_length,
    p_bound=p_bound,
    ARdegree=ar_degree,
    MAdegree=ma_degree,
    alpha=alpha,
    save_as_csv=True,
    destfolder=None,
    modelprefix='',
    track_method='',
    return_data=False
)

print('Filtrado completado exitosamente')

## 7. Creación de video con etiquetas

Generamos un video con las predicciones superpuestas para visualizar los resultados del seguimiento.

Nota: Ajuste la ruta `output_video` según su sistema antes de ejecutar esta celda.

In [ ]:
# Ruta al video que queremos etiquetar con las predicciones
output_video = 'experiments/A-7_crop.mp4'

print(f'Creando video etiquetado...')
print(f'Video de entrada: {output_video}')

# Creamos el video con las etiquetas y puntos clave marcados
# Este video muestra visualmente los resultados del análisis
dlc.create_labeled_video(
    path_config_file,
    [output_video],
    shuffle=1,
    trainingsetindex=0,
    filtered=False,
    save_frames=False,
    displayedbodyparts='all',
    draw_skeleton=False,
    trailpoints=0,
    videotype='mp4'
)

print('\nVideo etiquetado creado exitosamente')
print('  El video estará en la misma carpeta con el sufijo "_labeled"')